In [87]:
#only for ipython notebook
%load_ext autoreload
%autoreload 2
import warnings; warnings.simplefilter('ignore')
import ipywidgets as widgets
from IPython.display import display


#keep these imports
import pandas as pd
import geopandas
import numpy as np
from geopandas.tools import sjoin
import folium
import googlemaps
from folium.plugins import MarkerCluster
from branca.colormap import linear
import datetime

import model_2_funcs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
predictions_df=pd.read_csv('ems_calls_predict_mod2.csv')
predictions_df['region_id'] = predictions_df.zcta.astype(str)
predictions_df.set_index('region_id', inplace=True)
predictions_df.dtypes

zcta              int64
year              int64
month             int64
day_of_month      int64
hour_of_day       int64
day_of_year       int64
week_of_year      int64
day_of_week       int64
is_weekend         bool
Call_counts     float64
dtype: object

# Functions

In [89]:
def get_preds(file,month,day,hour):
    '''Get current predictions of hour from CSV file'''
    preds = pd.read_csv(file)
    pre1 = preds[(preds.month == month) &
             (preds.day_of_month == day) &
             (preds.hour_of_day == hour)
            ]
    pre1['prob'] = pre1.Call_counts/pre1.Call_counts.sum()
    return pre1

In [90]:
amb_status_file_path='./ambulance_loc.csv'
new_amb_locations = model_2_funcs.update_ambulance_assignments(amb_status_file_path=amb_status_file_path,
                                                               shape_file_path='sf_zcta/sf_zcta.shp',
                                                               predictions_file_path='ems_calls_predict_mod2.csv',
                                                               results_file_path=None,
                                                               shape_region_id_col = 'ZCTA5CE10')

region_id
94102    1.100496
94102    1.131471
94102    1.163319
94102    1.196062
94102    1.229728
94102    1.264341
94102    1.299928
94102    1.336517
94102    1.374136
94102    1.412813
94102    1.452580
94102    1.493465
94102    1.535501
94102    1.578721
94102    1.623157
94102    1.668844
94102    1.715817
94102    1.764112
94102    1.813766
94102    1.864818
94102    1.917306
94102    1.971273
94102    2.026758
94102    2.083805
94102    1.171763
94102    1.204745
94102    1.238654
94102    1.273519
94102    1.309364
94102    1.346219
           ...   
94102    1.828539
94102    1.880007
94102    1.932923
94102    1.987329
94102    2.043266
94102    2.100778
94102    1.114747
94102    1.146124
94102    1.178384
94102    1.211552
94102    1.245653
94102    1.280714
94102    1.316762
94102    1.353825
94102    1.391931
94102    1.431110
94102    1.471391
94102    1.512806
94102    1.555387
94102    1.599166
94102    1.644178
94102    1.690456
94102    1.738037
94102    1.786957


TypeError: cannot convert the series to <class 'float'>

In [8]:
def get_current_call(file):
    calls = pd.read_csv(file,parse_dates=[3]) 
    current_call = calls.loc[calls.TIME == max(calls.TIME)]
    return current_call.values[0]

def get_api_key(filepath):
    with open(filepath) as f:
        content = f.readlines()
    # remove whitespace characters at the end of each line
    content = [x.strip() for x in content]
    key = content[0]
    return key

def api_call(amb_coord,call_coord,dep_time,key,available_amb):
    gmaps = googlemaps.Client(key=key)
    result = gmaps.distance_matrix(amb_coord, call_coord, mode="driving", units="imperial", departure_time=dep_time)
    output_mat = pd.DataFrame()
    for idx, row in enumerate(result['rows']):
        row_mat = pd.DataFrame()
        mat = row['elements'][0]
        for key, val in mat.items():
            if key != 'status':
                df = pd.DataFrame.from_dict(val, orient='index')
                df = df.transpose()
                df.columns = [key + "_" + c for c in df.columns]
                if row_mat.empty:
                    row_mat = df
                else:
                    row_mat = pd.concat([row_mat, df], axis=1)
        if output_mat.empty:
            output_mat = row_mat
        else:
            output_mat = output_mat.append(row_mat)
    output_mat.index = [amb for amb in available_amb.AMB_ID]
    
    chosen = output_mat.loc[output_mat.duration_in_traffic_value == min(output_mat.duration_in_traffic_value)].index[0]
    return chosen

In [9]:
def add_ambmarker(smap,lat,long,amb_name):
    folium.Marker([lat, long], icon=folium.Icon(icon='plus',color='blue'),
                  popup="Amb #:" + amb_name
                 ).add_to(smap)
    
def dispatch_ambmarker(smap,lat,long,amb_name):
    folium.Marker([lat, long], icon=folium.Icon(icon='plus',color='gray'),
                  popup="Amb #:" + amb_name
                 ).add_to(smap)
    
def add_emsmarker(smap,lat,long,event_id):
    folium.RegularPolygonMarker([lat, long], popup="EMS #:" + event_id,
                                fill_color='red',number_of_sides=5,radius=10).add_to(smap)


In [10]:
def create_map(gfile,predsfile,ambfile,current_call):  
    #geopandas
    geodata = geopandas.read_file(gfile)
       
    #load in preds file for the hour of day of month
    t = get_preds(predsfile,datetime.datetime.now().month,
          datetime.datetime.now().day,datetime.datetime.now().hour)

    geodata['ZCTA5CE10'] = geodata['ZCTA5CE10'].astype('int64')
    gdf = geodata.merge(t,left_on='ZCTA5CE10' ,right_on='zcta')
    gdf1 = gdf
    gdf = gdf.set_index('ZCTA5CE10')['prob']
    gdf1 = gdf1.set_index('ZCTA5CE10')
    gdf1.crs={'init': 'epsg:4326'}
    
    #color scale
    colormap = linear.OrRd.scale(gdf1.prob.min(),gdf1.prob.max())
    
    #foliium
    sfmap = folium.Map([37.7556, -122.4399], zoom_start = 12)
    
    #plot zip codes and prob color grid
    folium.GeoJson(gdf1.to_json(),overlay=True,
        style_function=lambda feature: 
               {'color': "black",
               'weight':1.5,
               'fillColor': colormap(gdf[int(feature['id'])])}
              ).add_to(sfmap)
    folium.LayerControl().add_to(sfmap)
    
    #read in ambulance data, add markers
    
    ambulance = pd.read_csv(ambfile)
    
    for i in ambulance.values:
        if i[1] != 0:
            if i[3] == 1:
                add_ambmarker(sfmap,i[1], i[2],i[0])

            else:
                dispatch_ambmarker(sfmap,i[1], i[2],i[0])

    #input ems event markers
    add_emsmarker(sfmap,current_call[1],current_call[2],current_call[0])

    return sfmap

In [11]:
def dispatch_ambulance(ambfile,current_call,api_key):
    ambulance = pd.read_csv(ambfile)
    available_amb = ambulance.loc[ambulance.AVAILABLE == 1]
    amb_coord = [(row[2], row[3]) for row in available_amb.itertuples()]
    call_coord = [(current_call[1], current_call[2])]
    dep_time = current_call[3]
    result = api_call(amb_coord,call_coord,dep_time,api_key,available_amb)
    
    print("DISPATCHED AMBULANCE #:",result)
    #update ambulance pd and save to file
    ambulance.AVAILABLE[ambulance.AMB_ID==result] = 0
    ambulance.to_csv(ambfile,index=False)

In [12]:
def activate_ambulance(amb,ambfile):
    ambulance = pd.read_csv(ambfile)
    ambulance.AVAILABLE[ambulance.AMB_ID==amb] = 1
    ambulance.to_csv(ambfile,index=False)

In [13]:
def get_coordinates(key, addr):
    """Runs google maps geocoding api to return lat/long coords
    for a list of addresses.
    key: string (API key)
    addr: list of strings (addresses)"""
    gmaps = googlemaps.Client(key=key)
    coords = []
    for ad in addr:
        geocode_result = gmaps.geocode(ad)
        lat_long = geocode_result[0]['geometry']['location']
        # Add tuple with lat & long to coords output
        coords.append((lat_long['lat'], lat_long['lng']))
    return coords
        

In [14]:
def model_input(key_code,text,num):
    #current_call is an array([key, lat, long, Timestamp]
    p = get_coordinates(key_code,[text.value])
    test = np.array([str(num),p[0][0],p[0][1],pd.Timestamp(str(datetime.datetime.now()))])
    return test

# Keys

In [15]:
##must be changed for everyone's key
key = get_api_key('/Users/priyagupta/Desktop/key.txt')
key_code = get_api_key('/Users/priyagupta/Desktop/key_code.txt')

# Class Test

In [16]:
#reactivate ambulance
amb = widgets.Text()
display(amb)

def handle_amb_submit(sender):
    print("Now Active",amb.value)
    activate_ambulance(amb.value,'ambulance_loc.csv')
    
amb.on_submit(handle_amb_submit)
 

In [17]:
text= widgets.Text()
display(text)

In [21]:
test = model_input(key_code,text,111)
map_test = create_map('sf_zcta/sf_zcta.shp','ems_calls_predict_mod2.csv','ambulance_loc.csv',test)
map_test

In [19]:
dispatch_ambulance('ambulance_loc.csv',test,key)

DISPATCHED AMBULANCE #: amb_7


In [20]:
map_test = create_map('sf_zcta/sf_zcta.shp','ems_calls_predict_mod2.csv','ambulance_loc.csv',test)
map_test